## Cloud Farm Health Test

#### Specify list of farm nodes to test:

In [21]:
nodes = [
    "clfarm002",
    "clfarm003",
    "clfarm004",
    "clfarm005",
    "clfarm006",
    "clfarm007",
    "clfarm008",
    "clfarm009",
    "clfarm010",
    "clfarm011",
    "clfarm012",
    "clfarm013",
    "clfarm014",
]

good_freq_MHz = 1500
iterations = 100  # how many iterations to run for each model

In [26]:
import degirum as dg, mytools

cloud_token = mytools.get_token()  # get cloud API access token from env.ini file
cloud_zoo_url = mytools.get_cloud_zoo_url()  # get cloud zoo URL from env.ini file


# define function to run a single model batch prediction
def do_test(node):

    zoo = dg.connect(node, cloud_zoo_url, cloud_token)
    model_name = "yolo_v5m_coco--512x512_pruned_quant_n2x_orca_1"    

    # load model
    with zoo.load_model(model_name) as model:
        # configure model
        model.image_backend = "opencv"  # select OpenCV backend
        model.input_numpy_colorspace = "BGR"
        model._model_parameters.InputImgFmt = ["JPEG"]
        model.measure_time = True

        # prepare input frame
        frame = "./images/TwoCats.jpg"
        frame = model._preprocessor.forward(frame)[0]

        # define source of frames
        def source():
            for fi in range(iterations):
                yield frame

        model(frame)  # run model once to warm up the system

        # run batch prediction
        t = mytools.Timer()
        for res in model.predict_batch(source()):
            f_MHz = model.time_stats()["DeviceFrequency_MHz"].avg
            if f_MHz < good_freq_MHz:
                break

        return {
            "postprocess_type": model.output_postprocess_type,
            "elapsed": t(),
            "time_stats": model.time_stats(),
        }


# print results
def print_result(node, res):
    latency_ms = res["time_stats"]["FrameTotalDuration_ms"].avg
    iterations = res["time_stats"]["CoreInferenceDuration_ms"].cnt
    temp_C = res["time_stats"]["DeviceTemperature_C"].avg
    f_MHz = res["time_stats"]["DeviceFrequency_MHz"].avg
    fps = iterations / res["elapsed"]
    status = "Good" if f_MHz == good_freq_MHz else "OVERHEATED"
    print(f"{node:10} | {fps:4.1f} | {latency_ms:7.1f} | {temp_C:7.0f} | {f_MHz:9.0f} | {status:10}")

print(f"{'Node':10} | {'FPS':4} | {'Lat, ms':7} | {'Temp, C':7} | {'Freq, MHz':9} | {'Status':10}")
print(f"{'-'*60}")

results = {}
for node in nodes:
    results[node] = do_test(node)
    print_result(node, results[node])



Node       | FPS  | Lat, ms | Temp, C | Freq, MHz | Status    
------------------------------------------------------------
clfarm002  | 57.7 |   150.4 |      53 |      1500 | Good      
clfarm003  | 57.7 |   150.2 |      52 |      1500 | Good      
clfarm004  | 57.9 |   149.9 |      51 |      1500 | Good      
clfarm005  | 57.9 |   149.8 |      60 |      1500 | Good      
clfarm006  | 57.3 |   151.2 |      53 |      1500 | Good      
clfarm008  | 57.8 |   150.2 |      43 |      1500 | Good      
clfarm009  | 57.9 |   149.3 |      46 |      1500 | Good      
clfarm010  | 57.8 |   150.0 |      48 |      1500 | Good      
clfarm011  | 57.6 |   150.3 |      51 |      1500 | Good      
clfarm012  | 57.2 |   151.4 |      41 |      1500 | Good      
clfarm013  | 57.8 |   150.0 |      53 |      1500 | Good      
clfarm014  | 57.9 |   150.0 |      52 |      1500 | Good      
